# EDA IBM stock 1min ticks

## Takeouts -  december 29/2019
* The distribution of the difference of the time series has very long tails: see plot. This was expected; still very ugly.
* Created a very clean dataset with 'pastValues', 'currentValue', 'futureValue', 'deltaMinutes'. 
* The precision recall curve that we already have looks quite good: we trained one single time.

## Takeouts -  december 29/2019

## To do
* Plot a precision recall curve in which we do several (monthly/weekly) training bashes.
* Turn the code that produces the 'clean' data set into a more reusable one
* The current version of the target is not very realistic.
* Need a more elaborated way of looking at histograms: the tails don't let me see anything: those tails are a huge concern.
* Look out for sklearn methods to do crossvalidation in our setting: do not reinvent the wheel.

In [ ]:
cd ~/Desktop/MyProjects/moneyManager/

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import plot_precision_recall_curve

In [ ]:
# only loading some rows
names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
df = pd.read_csv('data/sampleKibotData/minuteIntraday/IBM_adjusted.txt', header=None, names=names)
df['DateTime'] = df.Date+' '+df.Time
df.DateTime = pd.to_datetime(df.DateTime,infer_datetime_format=True) 
ds = df[['DateTime', 'Open']].copy()

In [ ]:
ds.set_index('DateTime').plot(figsize=(15,5), title='IBM stock');

In [ ]:
ds.DateTime.apply(lambda x: x.year).value_counts().sort_index().plot.bar(figsize=(15, 5), title='ticks per year');

In [ ]:
ds.Open.diff().plot.hist(bins=200, figsize=(15,5), title= 'distribution of the differential');

In [ ]:
windowSize = 5
dsPast = pd.concat([ds.Open.shift(i) for i in range(windowSize, 0, -1)], axis=1).apply(lambda x: list(x), axis=1)
dsPresent = ds.Open
dsFuture = pd.concat([ds.Open.shift(i) for i in range(-1, -windowSize-1, -1)], axis=1).apply(lambda x: list(x), axis=1)
deltaMinutes = (ds.DateTime.shift(-windowSize) - ds.DateTime.shift(windowSize)).apply(lambda x: x.seconds)//60

In [ ]:
dg = pd.concat([ds.DateTime, dsPast, dsPresent, dsFuture, deltaMinutes], axis=1)
dg.columns = ['DateTime', 'pastValues', 'currentValue', 'futureValue', 'deltaMinutes']
dg = dg.set_index('DateTime')
dg['target']=(dg.futureValue.apply(lambda x: max(x))>dg.currentValue)
dg=dg[dg.deltaMinutes==10.0]
dg.head()

In [ ]:
dg.target.value_counts().plot.bar(title='distribution of (price is increasing in the near future)');

In [ ]:
X = pd.concat([dg.pastValues.apply(lambda x:pd.Series(x)), dg.currentValue, dg.target], axis=1)
y = X.pop('target')

In [ ]:
# rescaling the dataframe
X = X.div(X.currentValue, axis=0)
X.drop(columns='currentValue')

In [ ]:
# temporal split of train/test
trainPercentage = 20
testBegins = (len(X)*trainPercentage)//100
Xtrain, ytrain = X[:testBegins], y[:testBegins]
Xtest, ytest = X[testBegins:], y[testBegins:]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
parameters = {
    'min_samples_leaf' : [100, 1000, 2000],
    'max_depth': [50,100, 200],
    'n_estimators': [100, 200],   
    'max_features': ['sqrt']
}

regr = RandomForestClassifier(class_weight='balanced')
grid = GridSearchCV(regr, parameters, cv=3, scoring='average_precision', return_train_score=True)
grid.fit(Xtrain, ytrain)
model = grid.best_estimator_

In [ ]:
model

In [ ]:
ytrue = list(ytest)
ypredicted=model.predict_proba(Xtest)[:,1]

In [ ]:
base_rate = sum(ytest)/len(ytest)

In [ ]:
base_rate

In [ ]:
average_precision = average_precision_score(ytest, ypredicted)
disp = plot_precision_recall_curve(model, Xtest, ytest)
disp.ax_.set_title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision))
plt.plot([0, 1], [base_rate, base_rate]);

In [ ]:
pd.Series(Xtrain.index).plot.bar()